In [ ]:
from os import environ
from tempfile import mkdtemp

input_folder = environ.get(
    'CROSSCOMPUTE_INPUT_FOLDER', 'tests/standard/input')
output_folder = environ.get(
    'CROSSCOMPUTE_OUTPUT_FOLDER', mkdtemp())

In [ ]:
import subprocess

subprocess.run('pip install onsset pandas xlrd'.split())

In [ ]:
import pandas as pd
from os.path import join, splitext

specifications_path = join(output_folder, 'specifications.xlsx')
excel_writer = pd.ExcelWriter(specifications_path)
for file_name in [
    'specs_data.csv',
    'scenario_info.csv',
    'scenario_parameters.csv',
]:
    table = pd.read_csv(join(input_folder, file_name))
    base_name = splitext(file_name)[0]
    sheet_name = ''.join(_.title() for _ in base_name.split('_'))
    table.to_excel(excel_writer, sheet_name=sheet_name)
excel_writer.save()

In [ ]:
from os import makedirs
from os.path import join

details_folder = join(output_folder, 'details')
summaries_folder = join(output_folder, 'summaries')
try:
    makedirs(details_folder)
    makedirs(summaries_folder)
except OSError:
    pass

In [ ]:
from onsset.runner import calibration, scenario

cities_path = join(input_folder, 'cities.csv')
calibrated_specifications_path = join(output_folder, 'specifications.xlsx')
calibrated_cities_path = join(output_folder, 'cities.csv') 

calibration(
    specs_path=specifications_path,
    csv_path=cities_path,
    specs_path_calib=calibrated_specifications_path,
    calibrated_csv_path=calibrated_cities_path)
scenario(
    specs_path=calibrated_specifications_path,
    calibrated_csv_path=calibrated_cities_path,
    results_folder=details_folder,
    summary_folder=summaries_folder)

In [ ]:
from glob import glob
from os.path import basename, splitext

def collect_scenarios(target_path, source_folder):
    tables = []
    for path in glob(join(source_folder, '*.csv')):
        table = pd.read_csv(path)
        table['ScenarioName'] = basename(splitext(path)[0])
        columns = list(table.columns)
        columns.insert(0, columns.pop(columns.index('ScenarioName')))
        tables.append(table[columns])
    pd.concat(tables).to_csv(target_path, index=False)
    
collect_scenarios(join(output_folder, 'details.csv'), details_folder)
collect_scenarios(join(output_folder, 'summaries.csv'), summaries_folder)